##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 组合学习算法

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/federated/tutorials/composing_learning_algorithms"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## 准备工作

开始之前，请运行以下代码来确保您的环境已正确设置。如果未看到问候语，请参阅[安装](../install.md)指南查看说明。 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from typing import Callable

import tensorflow as tf
import tensorflow_federated as tff

**注**：本 Colab 已通过验证，可与[最新发布版本](https://github.com/tensorflow/federated#compatibility)的 `tensorflow_federated` pip 软件包一起使用，但 Tensorflow Federated 项目仍处于预发布开发阶段，可能无法在 `main` 上运行。

# 组合学习算法

[构建您自己的联合学习算法教程](https://github.com/tensorflow/federated/blob/v0.34.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)使用 TFF 的 Federated Core 直接实现联合平均 (FedAvg) 算法的一个版本。

在本教程中，您将使用 TFF 的 API 中的联合学习组件以模块化方式构建联合学习算法，而无需从头开始重新实现所有内容。

出于本教程的目的，您将实现能够在局部训练中使用梯度裁剪的 FedAvg 的变体。

## 学习算法构建块

概括来讲，许多学习算法都可以划分为四大独立组件，称为**构建块**。包括：

1. 分发器（即服务器到客户端的通信）
2. 客户端工作（即本地客户端计算）
3. 聚合器（即客户端到服务器的通信）
4. 终结器（即使用聚合客户端输出的服务器计算）

[构建您自己的联合学习算法教程](https://github.com/tensorflow/federated/blob/v0.34.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)从头开始实现了上述所有构建块，但通常不必这样做。相反，您可以重用相似算法的构建块。

在这种情况下，要实现采用梯度裁剪的 FedAvg，只需修改**客户端工作**构建块即可。其余块可以与普通 FedAvg 中使用块的相同。

# 实现客户端工作

首先，让我们编写采用梯度裁剪进行本地模型训练的 TF 逻辑。为简单起见，裁剪梯度的范数最大为 1。

## TF 逻辑

In [ ]:
@tf.function
def client_update(model: tff.learning.Model,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

关于上方代码，有几个要点。首先，它会跟踪所见样本的数量，因为这将构成客户端更新的*权重*（计算客户端的平均值时）。

其次，它会使用 [`tff.learning.templates.ClientResult`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/templates/ClientResult) 来包装输出。此返回类型用于标准化 `tff.learning` 中的客户端工作构建块。

## 创建 ClientWorkProcess

虽然上方的 TF 逻辑将采用裁剪进行本地训练，但仍需要将其包装在 TFF 代码中以创建必要的构建块。

具体而言，这 4 个构建块将表示为 [`tff.templates.MeasuredProcess`](https://tensorflow.google.cn/federated/api_docs/python/tff/templates/MeasuredProcess)。这意味着 4 个块均具有用于实例化和运行计算的 `initialize` 和 `next` 函数。

这使每个构建块都可以根据需要跟踪自己的**状态**（存储在服务器上）以执行其运算。虽然本教程中不予使用，但它可用于跟踪发生了多少次迭代，或跟踪优化器状态等用途。

客户端工作 TF 逻辑通常应包装为 [`tff.learning.templates.ClientWorkProcess`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/templates/ClientWorkProcess)，它可以编码进出客户端局部训练的预期类型。它可以由模型和优化器进行参数化，如下所示。

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.Model],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.framework.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# 组合学习算法

让我们将上方的客户端工作置于成熟的算法中。首先，让我们设置我们的数据和模型。

## 准备输入数据

加载和预处理包含在 TFF 中的 EMNIST 数据集。有关详情，请参阅[图像分类](federated_learning_for_image_classification.ipynb)教程。

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

为了将数据集馈送到我们的模型中，数据将被展平并转换为 `(flattened_image_vector, label)` 形式的元组。

让我们选择少量客户端，并将上述预处理应用于其数据集。

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 准备模型

此处使用与[图像分类](federated_learning_for_image_classification.ipynb)教程中相同的模型。此模型（通过 `tf.keras` 实现）具有一个隐藏层，随后是一个 softmax 层。为了在 TFF 中使用此模型，Keras 模型被包装为 [`tff.learning.Model`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model)。这使我们可以在 TFF 中执行模型的[前向传递](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model#forward_pass)，并[提取模型输出](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)。有关详情，另请参阅[图像分类](federated_learning_for_image_classification.ipynb)教程。

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 准备优化器

就像在 [`tff.learning.algorithms.build_weighted_fed_avg`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) 中一样，此处有两个优化器：客户端优化器和服务器优化器。为简单起见，优化器将为具有不同学习率的 SGD。

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## 定义构建块

客户端工作构建块、数据、模型和优化器现已设置完成，接下来还要为分发器、聚合器和终结器创建构建块。这可以通过借用 TFF 中可用并由 FedAvg 使用的一些默认值来完成。

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## 组合构建块

最后，您可以使用 TFF 中的内置**组合器**将构建块组合到一起。这是一种相对简单的组合器，采用上方的 4 个构建块并将它们的类型连接在一起。

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# 运行算法

算法现已就绪，让我们运行该算法。首先**初始化**该算法。该算法的**状态**对于每个构建块都有一个组件，以及一个用于*全局模型权重*的组件。

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

正如预期，客户端工作具有空状态（请记住上方的客户端工作代码！）。但是，其他构建块可能具有非空状态。例如，终结器会跟踪发生了多少次迭代。由于 `next` 尚未运行，它的状态为 `0`。

In [ ]:
state.finalizer

[0]

现在进行一轮训练。

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

此项的输出 (`tff.learning.templates.LearningProcessOutput`) 具有 `.state` 和 `.metrics` 输出。让我们看看这两者。

In [ ]:
learning_process_output.state.finalizer

[1]

显然，由于已经运行了一轮 `.next`，终结器状态递增 1。

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

虽然指标为空，但对于更加复杂和实际的算法，它们通常会包含大量实用信息。

# 结论

通过使用上述构建块/组合器框架，您无需从头开始重新处理所有内容，即可创建全新的学习算法。然而，这只是起点。这个框架使通过简单修改 FedAvg 的形式表达算法变得更加容易。有关更多算法，请参阅 [`tff.learning.algorithms`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms)，其中包含诸如 [FedProx](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) 和[客户端学习率调度 FedAvg](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule) 等算法。这些 API 甚至可以帮助实现诸如[联合 K 均值聚类](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans)等全新算法。